In [1]:
import pandas as pd
import altair as alt

In [55]:
auftraggeber = pd.read_csv('auftraggeber-neu.csv')

In [67]:
geber = auftraggeber.groupby('auftraggeber-per-name').count().sort_values('title', ascending=False).sort_index()
geber = geber[['title']]
geber.drop('Tschichold', inplace=True)
geber = geber[geber['title'] != 1]


In [64]:
geber.sort_index(inplace=True)

In [117]:
auftrag_per = alt.Chart(geber.reset_index()).mark_bar().encode(
        alt.Y('auftraggeber-per-name:N', sort='-x', title='Auftraggeber'),
        alt.X('title:Q', title='Datensätze'),
        tooltip=[alt.Tooltip('auftraggeber-per-name:N', title='Auftraggeber'), alt.Tooltip('title:Q', title='Anzahl')]
    )

text = auftrag_per.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='title:Q'
)

diagramm = alt.layer(auftrag_per, text).configure_view(
    stroke='transparent'
).configure_axis(
    domain=False
)

diagramm

alt.LayerChart(...)

In [80]:
with open('auftraggeber_personen.html', 'w') as f:
    f.write(diagramm.to_html())

In [71]:
inst_geber = auftraggeber.groupby('auftraggeber-inst-name').count().sort_values('title', ascending=False).sort_index()
inst_geber = inst_geber[['title']]
inst_geber = inst_geber[inst_geber['title'] != 1]

In [118]:
auftrag_inst = alt.Chart(inst_geber.reset_index()).mark_bar().encode(
        alt.Y('auftraggeber-inst-name:N', sort='-x', title='Auftraggeber'),
        alt.X('title:Q', title='Datensätze'),
        tooltip=[alt.Tooltip('auftraggeber-inst-name:N', title='Auftraggeber'), alt.Tooltip('title:Q', title='Anzahl Datensätze')]
    ).transform_window(
    rank='rank(title)',
    sort=[alt.SortField('title', order='descending')]
    ).transform_filter(
    (alt.datum.rank < 31))

text = auftrag_inst.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='title:Q'
)

diagramm = alt.layer(auftrag_inst, text).configure_view(
    stroke='transparent'
).configure_axis(
    domain=False
)

diagramm

alt.LayerChart(...)

In [95]:
with open('auftraggeber_inst.html', 'w') as f:
    f.write(diagramm.to_html())

In [46]:
entstehungsorte = pd.read_csv('entstehungsorte.csv')
orte = pd.read_csv('orte-geo.csv')

In [48]:
import geopy.geocoders as geo
from geopy.extra.rate_limiter import RateLimiter
geolocator = geo.Nominatim(user_agent='rundschreiben')
geo.options.default_timeout = 10
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=.5)


In [49]:
orte['location'] = orte['ort-name'].apply(geocode)
orte['lat'] = orte['location'].apply(lambda loc: loc.latitude if loc else None)
orte['lon'] = orte['location'].apply(lambda loc: loc.longitude if loc else None)

In [51]:
orte.to_csv('orte-geo-koord.csv', index=None)

In [53]:
entstehungsorte = entstehungsorte.merge(orte, on='ort-name')

In [68]:
ort_anzahl = pd.DataFrame(entstehungsorte.value_counts(['ort-name']).reset_index())
ort_anzahl.columns = ['ort-name','n']

In [72]:
orte_fin = orte.merge(ort_anzahl, on='ort-name')

In [117]:
import pydeck as pdk
#Karte
INITIAL_VIEW_STATE = pdk.ViewState(
    latitude=50.67877877706058,
    longitude=8.129981238464392,
    zoom=4.5,
    max_zoom=16,
    bearing=0,
    )

scatterplotlayer = pdk.Layer(
    "ScatterplotLayer",
    orte_fin[['ort-name','lat','lon','n']],
    pickable=True,
    opacity=0.5,
    stroked=True,
    filled=True,
    radius_min_pixels=2,
    line_width_min_pixels=1,
    get_position='[lon, lat]',
    get_radius="n*50",
    get_fill_color=[255, 140, 0],
    get_line_color=[0, 0, 0],
)

r = pdk.Deck(
    scatterplotlayer,
    initial_view_state=INITIAL_VIEW_STATE,
    map_style=pdk.map_styles.LIGHT
)

In [109]:
r.to_html()

In [96]:
objektgattungen = pd.read_csv('objektgattungen.csv')
gattungen = objektgattungen.groupby('idn').first().groupby('ts').count().reset_index()

In [119]:
gattungen_chart = alt.Chart(gattungen).mark_bar().encode(
        alt.Y('ts:N', sort='-x', title='Gattung'),
        alt.X('title:Q', title='Datensätze'),
        tooltip=[alt.Tooltip('ts:N', title='Gattung'), alt.Tooltip('title:Q', title='Anzahl Datensätze')]
    ).transform_window(
    rank='rank(title)',
    sort=[alt.SortField('title', order='descending')]
    ).transform_filter(
    (alt.datum.rank < 35))

text = gattungen_chart.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='title:Q'
)

diagramm = alt.layer(gattungen_chart, text).configure_view(
    stroke='transparent'
).configure_axis(
    domain=False
)

diagramm

alt.LayerChart(...)

In [100]:
with open('gattungen.html', 'w') as f:
    f.write(diagramm.to_html())

In [102]:
druckschriften = pd.read_csv('druckschriften_kurztitel.csv')

In [120]:
druckschriften_chart = alt.Chart(druckschriften).mark_bar().encode(
        alt.Y('schrift:N', sort='-x', title='Schrift'),
        alt.X('title:Q', title='Datensätze'),
        tooltip=[alt.Tooltip('schrift:N', title='Schrift'), alt.Tooltip('title:Q', title='Anzahl Datensätze')]
    )

text = druckschriften_chart.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='title:Q'
)
diagramm = alt.layer(druckschriften_chart, text).configure_view(
    stroke='transparent'
).configure_axis(
    domain=False
)

diagramm

alt.LayerChart(...)

In [106]:
with open('druckschriften.html', 'w') as f:
    f.write(diagramm.to_html())